In this notebook we'll be building our first LSTM model with the meteorological and site readings from the S.H Bell monitoring site. We'll first need to join all of the data and do some basic data cleaning and preparation.

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime